In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2025-01-06
Found rundate 2025-01-05 10:00:05 < 2025-01-06 at commit 15019486; returning commit 761494b


'2025-01-06 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

/home/runner/work/nj-crashes/nj-crashes/njsp/ytd.py:116: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: (


As of 2025-01-06 10:00:02, NJSP was reporting 2 YTD deaths
Current YTD Deaths (2026-01-06 10:01:17-05:00): 9
Previous year YTD Deaths (adjusted; 2025-01-06 10:00:02): 2.0003205114508056
Projected 2026 total: 612.8123520857091
349.9% change


/home/runner/work/nj-crashes/nj-crashes/njsp/ytd.py:129: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ytds = ytds.groupby('Year').apply(fill_all_days, rundate=self.rundate).reset_index()
3c25fde15ac5d08c3eaa4d70f97eca2d7913fa01: FAUQStats cache miss: 2025, Mon Jan 06 10:00:02 EST 2025


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,dt
ACCID,,,,,,,,,,,,
13682,12,Middlesex,1221,South Brunswick Tws,130,State Highway 130,1.0,0.0,1.0,0.0,0.0,2025-01-01 07:24:00-05:00
13686,11,Mercer,1103,Hamilton Twsp,95,New Jersey Turnpike MP 59.2,1.0,0.0,0.0,1.0,0.0,2025-01-02 22:01:00-05:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,1,0,0,0,1
Bergen,1,0,0,0,1
Gloucester,1,0,0,0,1
Middlesex,1,0,1,0,2
Morris,0,0,1,0,1
Passaic,1,1,0,0,1
Sussex,1,0,0,0,1


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

1.5% through the year, 98.5% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Mercer,driver,0,11,0
Middlesex,driver,0,27,1
Mercer,passenger,0,5,0
Middlesex,passenger,1,6,0
Mercer,pedestrian,1,8,0
...,...,...,...,...
Salem,crashes,0,18,0
Somerset,crashes,0,13,0
Sussex,crashes,0,11,1


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,34,0,20,4,12
Bergen,34,0,25,2,7
Burlington,38,1,24,6,10
Camden,38,2,20,3,15
Cape May,12,0,10,4,0
Cumberland,21,0,11,3,8
Essex,41,1,11,5,25
Gloucester,28,1,21,6,3
Hudson,20,2,8,1,11


In [9]:
projected.to_csv(PROJECTED_CSV)